In [31]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim

数据集准备

In [32]:
batch_size = 64  # 设置批处理大小
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])  # 定义数据预处理流程

train_dataset = datasets.MNIST(root='../datasets', train=True, download=True, transform=transform)  # 加载MNIST训练数据集
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)  # 创建训练数据加载器

test_dataset = datasets.MNIST(root='../datasets', train=False, download=True, transform=transform)  # 加载MNIST测试数据集
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)  # 创建测试数据加载器

定义模型

In [33]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        # 定义第一个全连接层，输入维度为784，输出维度为512
        self.fc1 = nn.Linear(784, 512)
        # 定义第二个全连接层，输入维度为512，输出维度为256
        self.fc2 = nn.Linear(512, 256)
        # 定义第三个全连接层，输入维度为256，输出维度为128
        self.fc3 = nn.Linear(256, 128)
        # 定义第四个全连接层，输入维度为128，输出维度为64
        self.fc4 = nn.Linear(128, 64)
        # 定义第五个全连接层，输入维度为64，输出维度为10
        self.fc5 = nn.Linear(64, 10)

    def forward(self, x):
        x = x.view(-1, 784)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        return self.fc5(x)
    



创建模型、损失函数、优化器

In [34]:
model = Model()

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

定义训练和预测方法

In [35]:
def train(epoch):
    running_loss = 0.0
    for batch_index, data in enumerate(train_loader, 0):
        inputs, labels = data
        optimizer.zero_grad()

        # 正向传播
        outputs = model(inputs)
        # 计算损失
        loss = criterion(outputs, labels)
        # 反向传播
        loss.backward()
        # 更新参数
        optimizer.step()

        # 打印300批的累计损失
        running_loss += loss.item()
        if batch_index % 300 == 299:
            print('[%d, %5d] loss: %.3f' % (epoch + 1, batch_index + 1, running_loss / 300))
            running_loss = 0.0

def test():
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            outputs = model(images)

            # 获取模型输出的最大值索引，用于预测类别，忽略掉最大值只获取最大值对应的索引
            _,predicted = torch.max(outputs.data, 1)

            # 统计预测正确的个数
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))


训练和预测

In [36]:
for i in range(5):
    train(i)
    test()
        

[1,   300] loss: 2.206
[1,   600] loss: 0.912
[1,   900] loss: 0.429
Accuracy of the network on the 10000 test images: 89 %
[2,   300] loss: 0.325
[2,   600] loss: 0.269
[2,   900] loss: 0.234
Accuracy of the network on the 10000 test images: 93 %
[3,   300] loss: 0.193
[3,   600] loss: 0.175
[3,   900] loss: 0.151
Accuracy of the network on the 10000 test images: 95 %
[4,   300] loss: 0.125
[4,   600] loss: 0.132
[4,   900] loss: 0.116
Accuracy of the network on the 10000 test images: 96 %
[5,   300] loss: 0.098
[5,   600] loss: 0.096
[5,   900] loss: 0.094
Accuracy of the network on the 10000 test images: 96 %
